In [12]:
from serpapi import GoogleSearch

def get_query(query):
    params = {
        "q": query,
        "engine": "google",
        "api_key": "d5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results['organic_results']
    return organic_results

organic_results = get_query("professor Lawrence Angrave")




In [13]:
import trafilatura as tr
import pandas as pd

def parse_results(link):
    downloaded = tr.fetch_url(link)
    text = tr.extract(downloaded)
    if text == None:
        return ''
    return text

def get_scholar_results(author_id):
    params = {
        'engine': 'google_scholar_author',
        'author_id': author_id,
        'api_key' : 'd5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f'
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

dataset = {'titles': [], 'text': []}

for result in organic_results:
    if 'scholar.google.com' in result['link']:
        # get user id from link
        user_id = result['link'].split('user=')[1].split('&')[0]
        # get scholar results
        scholar_results = get_scholar_results(user_id)
        print(scholar_results["author"])
    else:
        with open("Documents/" + result['title'] + '.txt', 'w') as f:
            f.write(parse_results(result['link']))
        dataset['titles'].append(result['title'])
        dataset['text'].append(parse_results(result['link']))

df = pd.DataFrame(dataset)
df.to_csv('Dataset.csv', index=False, sep='\t')

        


{'name': 'Lawrence Angrave', 'affiliations': 'Teaching Professor of Computer Science, University of Illinois at Urbana Champaign', 'email': 'Verified email at illinois.edu', 'thumbnail': 'https://scholar.google.com/citations/images/avatar_scholar_128.png'}


ERROR:trafilatura.utils:parsed tree length: 0, wrong data type or not valid HTML
ERROR:trafilatura.core:empty HTML tree for URL None
ERROR:trafilatura.utils:parsed tree length: 0, wrong data type or not valid HTML
ERROR:trafilatura.core:empty HTML tree for URL None


In [14]:
text = ''
with open('Documents/Lawrence Angrave Professor of Computer Science.txt', 'r') as f:
    text = f.read()
print(text)

Lawrence Angrave was a highly successful science student at RLS during the 1980’s. His Mum tells me that she drove him to Oxford University on the day the Berlin Wall came down. More recently, Lawrence has been working in the States and for about two years he’s been a Professor of Computer Science at the University of Illinois.His classes are large: Mum tells me that he has back to back classes of 260 students! To be fair, Lawrence has a team of staff working with him. Lawrence has received an award for teaching excellence, and here’s a typical rating from one of his students:
“One of the most charismatic teachers at the Universitiy. He loves his work, and he’ll make you love it too. While it was one of my trickier classes, it was the only class I loved to attend. Lectures are riddled with jokes and demonstrations. The difficult and abstract subject matter is made quite simple. Highly recommend Prof. Angrave.”
Well Done, Lawrence!
Leave a Reply


In [15]:

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base",max_length=2048)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565

KeyboardInterrupt: 

In [ ]:
input_text = "Answer based on context \n\n context: v1 \n\n question: v2" 
input_text = input_text.replace('v1', text)
input_text = input_text.replace('v2', 'who is professor lawrence angrave?')
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [10]:
from transformers import RagRetriever

dataset_path = 'knowledge_dir/my_knowledge_dataset/'
index_path = 'knowledge_dir/my_knowledge_dataset_hnsw_index.faiss'
retriever = RagRetriever.from_pretrained(
    "facebook/dpr-ctx_encoder-single-nq-base",
    index_name="custom",
    passages_path=dataset_path,
    index_path=index_path,
)



You are using a model of type dpr to instantiate a model of type rag. This is not supported for all configurations of models and can yield errors.


AssertionError: Config has to be initialized with question_encoder and generator config